In [1]:
name = 'CVMUPG-1206_purina'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *
from logic_filters import *
from segmentation import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

### Выбираем гостей нужного юзкейса

In [5]:
usecase = ['upgrade', 'cross', 'ump']
if datetime.datetime.today().time().hour > 12:
    dt = datetime.date.today()
else:
    dt = datetime.date.today() - datetime.timedelta(days=1)
lifetime = 90
freq = 1

In [6]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
                     .filter(F.col('lifetime') >= lifetime)
                     .filter(F.col('frequency') >= freq)
                     .select('customer_rk')
                            )


In [7]:
print(customers_usecase.count())

[Stage 0:=============================>                            (5 + 5) / 10]

16002346


### 1 проверяем на доступность отобранных гостей в определенную дату

In [8]:
check_date = '2022-03-15'

In [9]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:05


In [10]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:05


In [11]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:07


In [12]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [13]:
seg_sms.write.parquet('temp1206_1', mode='overwrite')
seg_sms = spark.read.parquet('temp1206_1')

In [14]:
seg_sms.count() #

8561002

In [15]:
seg_slip1 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:08


In [16]:
seg_slip2 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:06


In [17]:
seg_slip3 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:06


In [18]:
seg_slip = seg_slip1.union(seg_slip2).union(seg_slip3).distinct()

In [19]:
seg_slip.write.parquet('temp1206_2', mode='overwrite')
seg_slip = spark.read.parquet('temp1206_2')

In [20]:
seg_slip.count() #

13312622

### Проверка на доступность плю в магазинах

In [21]:
plu_codes = [3691531, 3469493, 3398569, 3468506, 3398568, 3468507, 3398572, 3398570, 3660972, 
      4009059, 4009065, 3641075] #POne_750_dry + #POne_wet


In [22]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
            .selectExpr('store_id as plant')
            .distinct()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

22105

### 1

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [50]:
start_date = datetime.date(2021, 9, 2)
end_date = datetime.date(2022, 3, 1)

In [51]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [52]:
checks_headers = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [53]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_slip, on='customer_rk', how='inner')

In [54]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [55]:
checks_headers_tc52 = checks_headers.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [56]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                                , 'zsale_vat'
                               )
                   )


In [57]:
plu_codes_cat = [3691531, 3469493, 3398569, 3468506, 3398568, 3468507, 3398572, 3398570, 3660972]

In [58]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [59]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [60]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [61]:
seg1 = (checks_tc5.withColumn('group', F.lit('sms'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [62]:
seg2 = (checks_tc52.withColumn('group', F.lit('slip'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [63]:
seg = seg1.union(seg2)

In [64]:
seg.write.parquet('temp1206_seg1', mode='overwrite')
seg = spark.read.parquet('temp1206_seg1')

In [65]:
seg_pd1 = seg.toPandas()

In [66]:
len(seg_pd1)

1123855

In [67]:
seg_no = 'seg1'

In [68]:
seg_pd1.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [69]:
seg_pd1['customer_rk'] = seg_pd1['customer_rk'].astype('str')

In [70]:
seg_pd1_sms = seg_pd1[seg_pd1.group == 'sms']
seg_pd1_slip = seg_pd1[seg_pd1.group == 'slip']

In [71]:
cross, upgrade, ump = get_usecases(seg_pd1_sms)

In [72]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 163362, upgrade: 190141, ump: 89025 (all: 442528)


In [73]:
cross, upgrade, ump = get_usecases(seg_pd1_slip)

In [74]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 271079, upgrade: 269241, ump: 141007 (all: 681327)


### 2 проверяем на доступность отобранных гостей в определенную дату

In [75]:
check_date = '2022-03-22'

In [76]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [77]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:04


In [78]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [79]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [80]:
seg_sms.write.parquet('temp1206_3', mode='overwrite')
seg_sms = spark.read.parquet('temp1206_3')

In [81]:
seg_sms.count() #

9128634

In [82]:
seg_slip1 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:01


In [83]:
seg_slip2 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:01


In [84]:
seg_slip3 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:01


In [85]:
seg_slip = seg_slip1.union(seg_slip2).union(seg_slip3).distinct()

In [86]:
seg_slip.write.parquet('temp1206_4', mode='overwrite')
seg_slip = spark.read.parquet('temp1206_4')

In [87]:
seg_slip.count() #

14342500

### Проверка на доступность плю в магазинах

In [88]:
plu_codes = [4125614, 3496903, 4125613, 3648595, 3677635, 4117910, 3496905, 3456543, 3456542, 3677636, 3456541,
            3496906, 3496902, 3496904, 4125673] #Friskies_50

In [89]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
            .selectExpr('store_id as plant')
            .distinct()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

23698

### 2

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [90]:
start_date = datetime.date(2021, 9, 2)
end_date = datetime.date(2022, 3, 1)

In [91]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [92]:
checks_headers = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [93]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_slip, on='customer_rk', how='inner')

In [94]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [95]:
checks_headers_tc52 = checks_headers.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [96]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                                , 'zsale_vat'
                               )
                   )


In [97]:
plu_codes_cat = [23881, 29522, 3677635, 3472951, 29521, 3648595, 25175, 4117910, 3472946, 4125673, 3469495, 3469498,
                 3496904, 3472948, 3652329, 3677636, 3472957, 4117909, 4125613, 3652327, 3472955, 3652328, 3472954, 
                 3456540, 3472949, 3474928, 21244, 3496908, 3469496, 4125614, 3456543, 3496902, 3900518, 3496906, 
                 3408783, 3496905, 3474927, 3496903, 3496901, 3456541, 3472953, 3945807, 3469494, 3456542, 32126]

In [98]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [99]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [100]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [101]:
seg1 = (checks_tc5.withColumn('group', F.lit('sms'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [102]:
seg2 = (checks_tc52.withColumn('group', F.lit('slip'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [103]:
seg = seg1.union(seg2)

In [104]:
seg.write.parquet('temp1206_seg2', mode='overwrite')
seg = spark.read.parquet('temp1206_seg2')

In [105]:
seg_pd2 = seg.toPandas()

In [106]:
len(seg_pd2)

5234149

In [107]:
seg_no = 'seg2'

In [108]:
seg_pd2.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [109]:
seg_pd2['customer_rk'] = seg_pd2['customer_rk'].astype('str')

In [110]:
seg_pd2_sms = seg_pd2[seg_pd2.group == 'sms']
seg_pd2_slip = seg_pd2[seg_pd2.group == 'slip']

In [111]:
cross, upgrade, ump = get_usecases(seg_pd2_sms)

In [112]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 760783, upgrade: 883702, ump: 381631 (all: 2026116)


In [113]:
cross, upgrade, ump = get_usecases(seg_pd2_slip)

In [114]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 1282121, upgrade: 1283313, ump: 642599 (all: 3208033)


### 3 проверяем на доступность отобранных гостей в определенную дату

In [7]:
check_date = '2022-04-05'

In [8]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:04


In [9]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [10]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [11]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [13]:
seg_sms.write.parquet('temp1206_5', mode='overwrite')
seg_sms = spark.read.parquet('temp1206_5')

[Stage 78:(2077 + 10) / 2087][Stage 94:(559 + 2) / 2087][Stage 102:(1883 + 2) / 2087]]]22/03/02 17:32:18 ERROR cluster.YarnScheduler: Lost executor 113 on mn-hd0089.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.
[Stage 112:(1086 + 10) / 2087][Stage 122:(2021 + 66) / 2087][Stage 128:(2066 + 21) / 2087]]22/03/02 17:35:00 ERROR cluster.YarnScheduler: Lost executor 190 on mn-hd0089.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.


In [14]:
seg_sms.count() #

8101383

In [15]:
seg_slip1 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:08


In [16]:
seg_slip2 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [17]:
seg_slip3 = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [18]:
seg_slip = seg_slip1.union(seg_slip2).union(seg_slip3).distinct()

In [20]:
seg_slip.write.parquet('temp1206_6', mode='overwrite')
seg_slip = spark.read.parquet('temp1206_6')

In [21]:
seg_slip.count() #

14044602

### Проверка на доступность плю в магазинах

In [22]:
plu_codes = [3691531, 3469493, 3398569, 3468506, 3398568, 3468507, 3398572, 3398570, 3660972, 
      4009059, 4009065, 3641075] #POne_750_dry + #POne_wet


In [23]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
            .selectExpr('store_id as plant')
            .distinct()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

22105

### 1

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [24]:
start_date = datetime.date(2021, 9, 2)
end_date = datetime.date(2022, 3, 1)

In [25]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [26]:
checks_headers = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [27]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_slip, on='customer_rk', how='inner')

In [28]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [29]:
checks_headers_tc52 = checks_headers.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [30]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                                , 'zsale_vat'
                               )
                   )


In [31]:
plu_codes_cat = [3691531, 3469493, 3398569, 3468506, 3398568, 3468507, 3398572, 3398570, 3660972]

In [32]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [33]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [34]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [35]:
seg1 = (checks_tc5.withColumn('group', F.lit('sms'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [36]:
seg2 = (checks_tc52.withColumn('group', F.lit('slip'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [37]:
seg = seg1.union(seg2)

In [38]:
seg.write.parquet('temp1206_seg3', mode='overwrite')
seg = spark.read.parquet('temp1206_seg3')

In [39]:
seg_pd3 = seg.toPandas()

In [40]:
len(seg_pd3)

1123781

In [41]:
seg_no = 'seg3'

In [42]:
seg_pd3.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [43]:
seg_pd3['customer_rk'] = seg_pd3['customer_rk'].astype('str')

In [44]:
seg_pd3_sms = seg_pd3[seg_pd3.group == 'sms']
seg_pd3_slip = seg_pd3[seg_pd3.group == 'slip']

In [45]:
cross, upgrade, ump = get_usecases(seg_pd3_sms)

In [46]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 154666, upgrade: 183152, ump: 75273 (all: 413091)


In [47]:
cross, upgrade, ump = get_usecases(seg_pd3_slip)

In [48]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 285491, upgrade: 286527, ump: 138672 (all: 710690)


### 4. Выбираем гостей нужного юзкейса

In [49]:
usecase = ['frequency']
if datetime.datetime.today().time().hour > 12:
    dt = datetime.date.today()
else:
    dt = datetime.date.today() - datetime.timedelta(days=1)
# lifetime = 90
# freq = 1

In [50]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
#                      .filter(F.col('lifetime') >= lifetime)
#                      .filter(F.col('frequency') >= freq)
                     .select('customer_rk')
                            )


In [51]:
print(customers_usecase.count())

[Stage 244:==================================================>     (9 + 1) / 10]

17295176


### 1 проверяем на доступность отобранных гостей в определенную дату

In [52]:
check_date = '2022-03-15'

In [53]:
seg_sms = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:07


In [54]:
seg_sms.write.parquet('temp1206_7', mode='overwrite')
seg_sms = spark.read.parquet('temp1206_7')

In [55]:
seg_sms.count() #

8021493

In [56]:
seg_slip = (slip_check_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False
                                        )
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [57]:
seg_slip.write.parquet('temp1206_8', mode='overwrite')
seg_slip = spark.read.parquet('temp1206_8')

In [58]:
seg_slip.count() #

14180851

### Проверка на доступность плю в магазинах

In [59]:
plu_codes = [3691531, 3469493, 3398569, 3468506, 3398568, 3468507, 3398572, 3398570, 3660972, 
      4009059, 4009065, 3641075] #POne_750_dry + #POne_wet


In [60]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
            .selectExpr('store_id as plant')
            .distinct()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

22105

### 4

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [61]:
start_date = datetime.date(2021, 9, 2)
end_date = datetime.date(2022, 3, 1)

In [62]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [63]:
checks_headers = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [64]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_slip, on='customer_rk', how='inner')

In [65]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [66]:
checks_headers_tc52 = checks_headers.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [67]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                                , 'zsale_vat'
                               )
                   )


In [68]:
plu_codes_cat = [3691531, 3469493, 3398569, 3468506, 3398568, 3468507, 3398572, 3398570, 3660972]

In [69]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [70]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [71]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [72]:
seg1 = (checks_tc5.withColumn('group', F.lit('sms'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [73]:
seg2 = (checks_tc52.withColumn('group', F.lit('slip'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [74]:
seg = seg1.union(seg2)

In [75]:
seg.write.parquet('temp1206_seg4', mode='overwrite')
seg = spark.read.parquet('temp1206_seg4')

In [76]:
seg_pd4 = seg.toPandas()

In [77]:
len(seg_pd4)

394124

In [78]:
seg_no = 'seg4'

In [79]:
seg_pd4.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [80]:
seg_pd4['customer_rk'] = seg_pd4['customer_rk'].astype('str')

In [81]:
seg_pd4_sms = seg_pd4[seg_pd4.group == 'sms']
seg_pd4_slip = seg_pd4[seg_pd4.group == 'slip']

In [82]:
print(len(seg_pd4_sms))
print(len(seg_pd4_slip))

140453
253671


22/03/03 00:47:46 ERROR cluster.YarnClientSchedulerBackend: YARN application has exited unexpectedly with state FAILED! Check the YARN application logs for more details.
22/03/03 00:47:46 ERROR cluster.YarnClientSchedulerBackend: Diagnostics message: Attempt recovered after RM restartDue to executor failures all available nodes are blacklisted
22/03/03 00:51:46 ERROR client.TransportClient: Failed to send RPC RPC 6207899545367401453 to /192.168.234.74:36638: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
22/03/03 00:51:46 ERROR cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Sending RequestExecutors(0,0,Map(),Set()) to AM was unsuccessful
java.io.IOException: Failed to send RPC RPC 6207899545367401453 to /192.168.234.74:36638: java.nio.channels.ClosedChannelException
	at org.apache.spark.network.client.TransportClient$RpcChannelListener.handleFailure(TransportClient.jav